In [5]:
from pomegranate import *

sdist = DiscreteDistribution({'wet': .50, 'dry': .50})

# This is the distribution for my water node
wdist = ConditionalProbabilityTable(
    [['wet', 'none', 0.8],
     ['wet', 'some', 0.15],
     ['wet', 'lots', 0.05],
     ['dry', 'none', 0.95],
     ['dry', 'some', 0.035],
     ['dry', 'lots', 0.015]], [sdist])

# Distribution for my water detector node
detectordist = ConditionalProbabilityTable(
    [['none', 'on', 0.01],
     ['none', 'off', 0.99],
     ['some', 'on', 0.90],
     ['some', 'off', 0.10],
     ['lots', 'on', 0.995],
     ['lots', 'off', .005]], [wdist])

#  My two nodes
season = Node(sdist, "season")
water = Node(wdist, "water")
waterDetector = Node(detectordist, "waterDetector")

# The Network
model = BayesianNetwork("Water Detector")
model.add_states(season, water, waterDetector)
model.add_edge(season, water)
model.add_edge(water, waterDetector)
model.bake()
           
def nodeIndex(model, nodeName, probs):
    return list(map(lambda s: s.name, model.states)).index(nodeName)

def probDist(nodeName, model, evidence):
    return model.predict_proba(evidence)[nodeIndex(model, nodeName, model)].parameters[0]

In [9]:
# Suppose I no information (including the season).  
# What is the likelihood that there is any water in my basement?  
# (I.e. water is not some and not lots)  Answer (single number!) in this cell.
probDist('water', model, {})['none']

0.8749999999999992

In [10]:
# Suppose it's the dry season, but my water detector is saying **on** -- 
# what do I believe about water in the basement? Answer in this cell -- three numbers,
# a distribution over the domain values (none, some, lots)
probDist('water', model, {'season': 'dry', 'waterDetector': 'on'})

{'lots': 0.2668752793920448,
 'some': 0.5632543585158643,
 'none': 0.16987036209209097}

#### Problem 2

In [16]:
bdist = DiscreteDistribution({True: .001, False: .999})
edist = DiscreteDistribution({True: .002, False: .998})
adist = ConditionalProbabilityTable(
    [[True, True, True, .95],
     [True, True, False, .05],
     [True, False, True, .94],
     [True, False, False, .06],
     [False, True, True, .29],
     [False, True, False, .71],
     [False, False, True, .001],
     [False, False, False, .999]], [bdist, edist])
jdist = ConditionalProbabilityTable(
    [[True, True, .9],
    [True, False, .1],
    [False, True, .45],
    [False, False, .55]], [adist])
mdist = ConditionalProbabilityTable(
    [[True, True, .7],
    [True, False, .3],
    [False, True, .01],
    [False, False, .99]], [adist])

#  My two nodes
b = Node(bdist, "burglary")
e = Node(edist, "earthquake")
a = Node(adist, "alarm")
j = Node(jdist, "john")
m = Node(mdist, "mary")

# The Network
model = BayesianNetwork("Alarm")
model.add_states(b,e,a,j,m)
model.add_edge(b, a)
model.add_edge(e,a)
model.add_edge(a, j)
model.add_edge(a, m)

model.bake()

In [17]:
print(probDist('burglary', model, {})[True])
print(probDist('alarm', model, {})[True])

0.001000000000000444
0.002516442000000935


In [19]:
b1 = probDist('burglary', model, {})[True]
b2 = probDist('burglary', model, {'john': False, 'mary': True})[True]
(b1, b2, (b2 - b1)/b1)

(0.001000000000000444, 0.011679205846255711, 10.679205846250527)

In [20]:
print(probDist('john', model, {'earthquake': True})[True])

0.580797
